<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Создание-лемматизационной-функции" data-toc-modified-id="Создание-лемматизационной-функции-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Создание лемматизационной функции</a></span></li><li><span><a href="#Формирование-набора-признаков-и-целевого-признака." data-toc-modified-id="Формирование-набора-признаков-и-целевого-признака.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Формирование набора признаков и целевого признака.</a></span></li><li><span><a href="#Разделение-датасета-на-обучающую-и-тестовую-выборки" data-toc-modified-id="Разделение-датасета-на-обучающую-и-тестовую-выборки-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Разделение датасета на обучающую и тестовую выборки</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-логистической-регрессии" data-toc-modified-id="Модель-логистической-регрессии-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель логистической регрессии</a></span></li><li><span><a href="#Модель-решающего-дерева" data-toc-modified-id="Модель-решающего-дерева-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель решающего дерева</a></span></li><li><span><a href="#CatBoost-Classifier" data-toc-modified-id="CatBoost-Classifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoost Classifier</a></span></li><li><span><a href="#Итоговая-таблица" data-toc-modified-id="Итоговая-таблица-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Итоговая таблица</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
from tqdm import tqdm

from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import f1_score

from datetime import datetime

import warnings
warnings.simplefilter("ignore")

In [2]:
try:
    df = pd.read_csv('datasets/toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')
df.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Всего в нашем датасете имеется 159,292 записей (строк) с двумя признаками:

- "text" - содержит текстовое содержание комментария без пропусков;
- "toxic" - бинарный признак, где значение 1 указывает на наличие токсичного комментария, а значение 0 - на отсутствие токсичности, пропусков в данных нет.

### Создание лемматизационной функции

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kirillkhrustitskii/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kirillkhrustitskii/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/kirillkhrustitskii/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kirillkhrustitskii/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kirillkhrustitskii/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
lemmatizer = WordNetLemmatizer()
tqdm.pandas()

Разработаем функцию, которая будет отвечать за лемматизацию и очистку текста

In [6]:
def get_wordnet_pos(word):
    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [7]:
def clear_text(text):
    word_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])
    return lemm_text

Применим данную функцию к набору данных.

In [8]:
df['lemm_text'] = df['text'].progress_apply(lambda x: clear_text(x))

100%|██████████████████████████████████| 159292/159292 [17:53<00:00, 148.39it/s]


In [9]:
df

,Unnamed: 0,text,toxic,lemm_text
0,0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,D'aww ! He match this background colour I 'm s...
2,2,"Hey man, I'm really not trying to edit war. It...",0,"Hey man , I 'm really not try to edit war . It..."
3,3,"""\nMore\nI can't make any real suggestions on ...",0,`` More I ca n't make any real suggestion on i...
4,4,"You, sir, are my hero. Any chance you remember...",0,"You , sir , be my hero . Any chance you rememb..."
...,...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0,"`` : : : : : And for the second time of ask , ..."
159288,159447,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That be a ho...
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0,"Spitzer Umm , there no actual article for pros..."
159290,159449,And it looks like it was actually you who put ...,0,And it look like it be actually you who put on...


### Формирование набора признаков и целевого признака.

Разделим признаки из датасета и целевой признак и сохраним их в отдельные переменные.

In [10]:
corpus = df['lemm_text']
target = df['toxic']

Преобразуем обучающие признаки в формат Unicode.

### Разделение датасета на обучающую и тестовую выборки

In [12]:
df_train, df_last = train_test_split(df, test_size=0.2, random_state=12345)

In [13]:
df_valid, df_test = train_test_split(df_last, test_size=0.5, random_state=12345)

Для оценки сбалансированности датасета, используем метод value_counts() для анализа распределения значений в целевом признаке.

In [14]:
df_train['toxic'].value_counts()

0    114477
1     12956
Name: toxic, dtype: int64

В данных замечается существенный дисбаланс классов, с преобладанием нулевого класса. Для коррекции этой проблемы, необходимо применить метод upsampling.

Разделим объекты по классам и сохраним их в отдельные переменные

In [15]:
df_nulls = df_train[df_train['toxic'] == 0]
df_ones = df_train[df_train['toxic'] == 1]

In [16]:
df_nulls_downsampled  = resample(df_nulls, replace=True, n_samples=df_train['toxic'].value_counts()[1]*2, random_state=12345)

Объединим полученные выборки

In [17]:
df_train = pd.concat([df_nulls_downsampled, df_ones])

Проверяем результат

In [18]:
df_train['toxic'].value_counts()

0    25912
1    12956
Name: toxic, dtype: int64

Отделим обучающие признаки от целевого признака

In [19]:
corpus_train = df_train['lemm_text']
corpus_valid = df_valid['lemm_text']
corpus_test = df_test['lemm_text']

target_train = df_train['toxic']
target_valid = df_valid['toxic']
target_test = df_test['toxic']

Для создания новых признаков для моделей рассчитаем и добавим в датасет TF-IDF значения. При этом также определим стоп-слова, которые будут исключены из анализа.

In [20]:
stopwords = nltk_stopwords.words('english')

count_tf_idf = TfidfVectorizer(min_df=2, stop_words=stopwords)
count_tf_idf.fit(corpus_train)

TfidfVectorizer(min_df=2,
                stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

Сформируем выборки обучающих, валидационных и тестовых признаков

In [21]:
features_train = count_tf_idf.transform(corpus_train).toarray()
features_valid = count_tf_idf.transform(corpus_valid).toarray()
features_test = count_tf_idf.transform(corpus_test).toarray()

features_train.shape, features_valid.shape, features_test.shape

((38868, 29895), (15929, 29895), (15930, 29895))

## Обучение

### Модель логистической регрессии

Обучим модель логистической регрессии

In [22]:
%%time
start_time = datetime.now()
model_lr = (LogisticRegression(solver='liblinear', random_state=12345).fit(features_train, target_train))
time_lr = datetime.now() - start_time
print(str(time_lr))

0:00:13.080217
CPU times: user 6.21 s, sys: 4.71 s, total: 10.9 s
Wall time: 13.1 s


Получим предсказания на тренировочной выборке и вычислим метрику F1.

In [23]:
predictions = model_lr.predict(features_valid)
f1_valid_lr = f1_score(target_valid, predictions)
f1_valid_lr

0.7528494301139773

### Модель решающего дерева

Подбираем оптимальные гиперпараметры модели

In [25]:
%%time
best_model_tree = None
best_result_tree = 0
best_depth_tree = 0

for depth in range(1, 3):
    model_tree = DecisionTreeClassifier(max_depth = depth,  random_state = 12345)
    model_tree.fit(features_train, target_train)
    prediction_tree_valid = model_tree.predict(features_valid)
    f1_tree = f1_score(target_valid, prediction_tree_valid)

    if f1_tree > best_result_tree:
        best_model_tree = model_tree
        best_result_tree = f1_tree
        best_depth_tree = depth

best_result_tree

CPU times: user 54.4 s, sys: 8.04 s, total: 1min 2s
Wall time: 1min 5s


0.4307692307692308

Обучаем модель на подобранных гиперпараметрах

In [26]:
%%time
start_time = datetime.now()
model_tree = DecisionTreeClassifier(max_depth = best_depth_tree,  random_state = 12345)
model_tree.fit(features_train, target_train)
time_tree = datetime.now() - start_time
print(str(time_tree))

0:00:40.508420
CPU times: user 35.3 s, sys: 3.33 s, total: 38.7 s
Wall time: 40.5 s


### CatBoost Classifier

Обучим модель CatBoost Classifier

In [28]:
%%time
start_time = datetime.now()
model_cat = CatBoostClassifier(loss_function= 'Logloss', iterations=400, eval_metric='F1', random_state=12345)
model_cat.fit(features_train, target_train, verbose=25)
prediction = model_cat.predict(features_test)
time_cat = datetime.now() - start_time
print(str(time_cat))

Learning rate set to 0.11392
0:	learn: 0.5363104	total: 255ms	remaining: 1m 41s
25:	learn: 0.6876805	total: 5.61s	remaining: 1m 20s
50:	learn: 0.7438354	total: 10.8s	remaining: 1m 13s
75:	learn: 0.7673242	total: 15.5s	remaining: 1m 5s
100:	learn: 0.7895526	total: 20.4s	remaining: 1m
125:	learn: 0.8068685	total: 25.2s	remaining: 54.8s
150:	learn: 0.8190384	total: 30.2s	remaining: 49.8s
175:	learn: 0.8301556	total: 34.8s	remaining: 44.3s
200:	learn: 0.8384709	total: 39.4s	remaining: 39s
225:	learn: 0.8445484	total: 43.8s	remaining: 33.8s
250:	learn: 0.8495667	total: 48.6s	remaining: 28.8s
275:	learn: 0.8560401	total: 53.4s	remaining: 24s
300:	learn: 0.8598179	total: 58.1s	remaining: 19.1s
325:	learn: 0.8654124	total: 1m 3s	remaining: 14.3s
350:	learn: 0.8700153	total: 1m 8s	remaining: 9.54s
375:	learn: 0.8730582	total: 1m 13s	remaining: 4.67s
399:	learn: 0.8753381	total: 1m 17s	remaining: 0us
0:07:03.365389
CPU times: user 13min 3s, sys: 15.5 s, total: 13min 18s
Wall time: 7min 3s


На тренировочной выборке и вычислим метрику F1.

In [29]:
f1_train_cat = model_cat.get_best_score()['learn']['F1']
f1_train_cat

0.8753380662609871

### Итоговая таблица

In [31]:
table_final = pd.DataFrame({'Модель': ['LogisticRegression', 'DecisionTreeClassifier', 'CatBoostClassifier'], 'F1_valid': [f1_valid_lr, best_result_tree, f1_train_cat], 'Скорость обучения': [time_lr, time_tree, time_cat]})

table_final

,Модель,F1_valid,Скорость обучения
0,LogisticRegression,0.752849,0 days 00:00:13.080217
1,DecisionTreeClassifier,0.430769,0 days 00:00:40.508420
2,CatBoostClassifier,0.875338,0 days 00:07:03.365389


Обучены несколько моделей машинного обучения, включая Логистическую Регрессию, Решающее Дерево и CatBoost. Для каждой из них получены предсказания на валидационной выборке. CatBoost показал самое высокое качество. 

Получим предсказания на тестовой выборке и вычислим метрику F1

In [32]:
f1_test_cat = f1_score(target_test, prediction)
f1_test_cat

0.7727558066541116

## Выводы

В данной работе были загружены необходимых библиотек, подготовлены данные, произведена балансировка классов, лемматизации текста, создания корпусов и разделения датасета на выборки.

Затем обучены несколько моделей машинного обучения, включая Логистическую Регрессию, Решающее Дерево и CatBoost. Для каждой из них получены предсказания на тестовой выборке. По полученным результатам можно сделать следующие выводы:

- Логистическая Регрессия, несмотря на свою простоту, продемонстрировала высокую метрику и быстрое обучение.
- Модель Решающего Дерева оказалась наименее подходящей для классификации текстов из-за длительного времени обучения, явного переобучения и низкой метрики.
- CatBoost показал самое высокое качество, но требует значительно больше времени для обучения по сравнению с Логистической Регрессией.

Таким образом, рекомендуется использовать модель CatBoost в разрабатываемом продукте.